In [1]:
# Import modules
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from functools import *
from google.colab import files

%tensorflow_version 1.x
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


TensorFlow 1.x selected.


In [2]:
uploaded = files.upload()
file_name = "bm25.csv"

Saving bm25.csv to bm25.csv


### Reading Data and Prepare Dataset

In [3]:
dataset = pd.read_csv(file_name, delimiter=',')
print(dataset.shape)
dataset.head()

(40, 4)


,DocumentID,DocumentLength,FrequencyOfWord1,FrequencyOfWord2
0,1,7,1,1
1,2,17,2,4
2,3,8,3,1
3,4,25,5,5
4,5,11,2,3


In [0]:
# Convert Pandas Dataframe to Numpy dArray
dataset_np = dataset.values

### Define BM25 function

In [0]:
import math
k = 1.2
b = 0.75

# Calculate IDF
def IDF(q):
    return math.log((dataset_np.shape[0] - DF(q) + 0.5) / (DF(q) + 0.5))

# Calculate DF: get the number of documents have that word
def DF(q):
    total = 0
    for index in range(dataset_np.shape[0]):
        if dataset_np[index, q] > 0:
            total += 1
    return total

# -------------
# Calculate :L average
def L():
    return np.average(dataset_np[:, 1])


# Calculate TF: get the frequency
def TF(documentIndex, q):
    return dataset_np[documentIndex, q]

# Calculate
# di: documentIndex
def BM25(di):
    total = 0
    for index in range(2):
        q = index + 2
        top = IDF(q) * ((TF(di, q) * (k+1)))
        bottom = TF(di, q) + k * (1 - b + b * (np.absolute(dataset_np[di, 1]) / L()))
        total += (top/bottom)
    return total

In [35]:
# Getting BM25 score for document number 6
di = 5
for i in range(2):
    q = i + 2
    top = IDF(q) * ((TF(di, q) * (k+1)))
    bottom = TF(di, q) + k * (1 - b + b * (np.absolute(dataset_np[di, 1]) / L()))
    print(top / bottom)


-0.0
-0.9251384415779851


In [0]:
BM25arr = []
for index in range(dataset_np.shape[0]):
    BM25arr.append(BM25(index))

BM25arrnp = np.array(BM25arr)
BM25Dataset = np.column_stack((dataset_np, BM25arrnp))

### Ranking

In [42]:
sortedBM25 = BM25Dataset[BM25Dataset[:, 4].argsort()]
print(sortedBM25)
print("Lowest scored document[%s]: %s" % (sortedBM25[0, 0], sortedBM25[0, 4]))
print("Highest scored document[%s]: %s" % (sortedBM25[-1, 0], sortedBM25[-1, 4]))

[[ 4.         25.          5.          5.         -2.04352403]
 [29.         22.          4.          5.         -2.02757522]
 [10.         12.          2.          4.         -1.98438757]
 [12.          8.          2.          2.         -1.98023121]
 [ 5.         11.          2.          3.         -1.96271982]
 [ 9.         13.          3.          2.         -1.94302782]
 [ 3.          8.          3.          1.         -1.92278601]
 [19.         14.          2.          3.         -1.87753887]
 [ 2.         17.          2.          4.         -1.85779173]
 [ 7.         12.          3.          1.         -1.79245306]
 [40.         20.          2.          3.         -1.72792642]
 [ 1.          7.          1.          1.         -1.6235557 ]
 [35.         25.          4.          1.         -1.57280308]
 [18.          8.          5.          0.         -1.37617836]
 [17.         27.          1.          3.         -1.31383467]
 [25.         15.          5.          0.         -1.29